In [1]:
from pickle import dump as pk_dump
from pickle import load as pk_load
from datetime import datetime
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json
from pathlib import Path
import pandas as pd

In [7]:
cloud_config= {'secure_connect_bundle': r'secure-connect-henry.zip'}
auth_provider = PlainTextAuthProvider(json.load(open(r'log_in.json'))['log_user'], json.load(open(r'log_in.json'))['log_password'])
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

In [30]:
reload_time = datetime.now().minute%5 == 0
reload_time = True

In [4]:
def cql_to_pandas(cql_query,cassandra_session):
    def pandaspark_factory(colnames, rows):\
        return pd.DataFrame(rows, columns=colnames)
    cassandra_session.row_factory = pandaspark_factory
    cassandra_session.default_fetch_size = None
    result = cassandra_session.execute(cql_query, timeout=None)
    return result._current_rows

In [29]:
def unique_business_id():
    if reload_time:
        unique = pd.concat([pd.Series(['tYCok-NtWvg8_k7woeB83w','Wr2k0Vz8RbcumYulp-jIrA','QKFojAIRYfQQzwssuoKjzw']), cql_to_pandas("""SELECT DISTINCT business_id FROM yelp.business""",session)['business_id']])
        pk_dump(unique, open(r'unique_business_id.pkl',"wb"))
        return unique
    else: return pk_load(open(r'unique_business_id.pkl',"rb"))['business_id']


In [33]:
unique_business_id()

0      tYCok-NtWvg8_k7woeB83w
1      Wr2k0Vz8RbcumYulp-jIrA
2      QKFojAIRYfQQzwssuoKjzw
0      61oyIzeFjG977kMhMxVznw
1      Dtuf4vD0znzwJW9vk3iTNw
                ...          
995    4dVrWc4Nhuw9apfk9BWEcA
996    ARb-hdkQ7Qla0FKCo4SPcA
997    VeFfrEZ4iWaecrQg6Eq4cg
998    Cyy0zZz5ixYjSX4o4sacpg
999    TapXvuOEC3qGGmlf03INnA
Length: 1003, dtype: object

0    tYCok-NtWvg8_k7woeB83w
1    Wr2k0Vz8RbcumYulp-jIrA
2    QKFojAIRYfQQzwssuoKjzw
dtype: object

0      61oyIzeFjG977kMhMxVznw
1      Dtuf4vD0znzwJW9vk3iTNw
2      luGY5CJlb4CJ3LrBcSXMPA
3      jcL_qaGJiappzpnn-ifSoA
4      feVi5a5xZaahs4WsUfi1tw
                ...          
995    4dVrWc4Nhuw9apfk9BWEcA
996    ARb-hdkQ7Qla0FKCo4SPcA
997    VeFfrEZ4iWaecrQg6Eq4cg
998    Cyy0zZz5ixYjSX4o4sacpg
999    TapXvuOEC3qGGmlf03INnA
Name: business_id, Length: 1000, dtype: object